In [1]:
%matplotlib inline
import sys, os
sys.path.append('../..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [2]:
from sktime.base import load
#from sklearn.model_selection import GridSearchCV
#from sktime.classification.feature_based import TSFreshClassifier
from sktime.classification.deep_learning.cnn import CNNClassifier

In [3]:
period = 100
filepath = os.path.join('..','..','data','trend index','training set', f'trend_{period}day.hdf')
file = h5py.File(filepath, mode='r')

In [5]:
ohlc  = file['ohlc'][:]
trend = file['trend'][:]
# shuffling
idx = np.random.permutation(len(ohlc))
ohlc = ohlc[idx]
trend = trend[idx]

print(ohlc.shape)
print(trend.shape)


(1021, 4, 100)
(1021,)


In [6]:
file.close()

### 1. Train data & Test data 생성 (7:3)

In [7]:
idx = int(len(ohlc)*0.7)
ohlc_train = ohlc[:idx]
trend_train = trend[:idx]

ohlc_test = ohlc[idx:]
trend_test = trend[idx:]

print(ohlc_train.shape)
print(ohlc_test.shape)

(714, 4, 100)
(307, 4, 100)


### 2. 훈련 모델 생성 및 훈련

In [9]:
network = CNNClassifier(n_epochs=200,batch_size=4, verbose=True)
#network = TSFreshClassifier(verbose=True)
network.fit(ohlc_train, trend_train)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 4)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 94, 6)             174       
                                                                 
 average_pooling1d_2 (Averag  (None, 31, 6)            0         
 ePooling1D)                                                     
                                                                 
 conv1d_3 (Conv1D)           (None, 25, 12)            516       
                                                                 
 average_pooling1d_3 (Averag  (None, 8, 12)            0         
 ePooling1D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 96)                0   

C:\Users\yosep\python\miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


179/179 [==============================] - 0s 910us/step - loss: 0.1751 - accuracy: 0.5658
Epoch 2/200
179/179 [==============================] - 0s 937us/step - loss: 0.1470 - accuracy: 0.6751
Epoch 3/200
179/179 [==============================] - 0s 948us/step - loss: 0.1405 - accuracy: 0.6877
Epoch 4/200
179/179 [==============================] - 0s 914us/step - loss: 0.1296 - accuracy: 0.7311
Epoch 5/200
179/179 [==============================] - 0s 918us/step - loss: 0.1271 - accuracy: 0.7311
Epoch 6/200
179/179 [==============================] - 0s 926us/step - loss: 0.1137 - accuracy: 0.7689
Epoch 7/200
179/179 [==============================] - 0s 920us/step - loss: 0.1164 - accuracy: 0.7437
Epoch 8/200
179/179 [==============================] - 0s 922us/step - loss: 0.1047 - accuracy: 0.8081
Epoch 9/200
179/179 [==============================] - 0s 914us/step - loss: 0.1001 - accuracy: 0.7941
Epoch 10/200
179/179 [==============================] - 0s 917us/step - loss: 0.0941 

Epoch 80/200
179/179 [==============================] - 0s 922us/step - loss: 0.0317 - accuracy: 0.9538
Epoch 81/200
179/179 [==============================] - 0s 916us/step - loss: 0.0309 - accuracy: 0.9580
Epoch 82/200
179/179 [==============================] - 0s 914us/step - loss: 0.0305 - accuracy: 0.9496
Epoch 83/200
179/179 [==============================] - 0s 914us/step - loss: 0.0301 - accuracy: 0.9622
Epoch 84/200
179/179 [==============================] - 0s 872us/step - loss: 0.0324 - accuracy: 0.9454
Epoch 85/200
179/179 [==============================] - 0s 925us/step - loss: 0.0283 - accuracy: 0.9622
Epoch 86/200
179/179 [==============================] - 0s 936us/step - loss: 0.0298 - accuracy: 0.9538
Epoch 87/200
179/179 [==============================] - 0s 937us/step - loss: 0.0273 - accuracy: 0.9608
Epoch 88/200
179/179 [==============================] - 0s 937us/step - loss: 0.0304 - accuracy: 0.9524
Epoch 89/200
179/179 [==============================] - 0s 935us

179/179 [==============================] - 0s 930us/step - loss: 0.0168 - accuracy: 0.9734
Epoch 159/200
179/179 [==============================] - 0s 936us/step - loss: 0.0129 - accuracy: 0.9832
Epoch 160/200
179/179 [==============================] - 0s 937us/step - loss: 0.0126 - accuracy: 0.9832
Epoch 161/200
179/179 [==============================] - 0s 936us/step - loss: 0.0211 - accuracy: 0.9678
Epoch 162/200
179/179 [==============================] - 0s 937us/step - loss: 0.0154 - accuracy: 0.9790
Epoch 163/200
179/179 [==============================] - 0s 936us/step - loss: 0.0122 - accuracy: 0.9818
Epoch 164/200
179/179 [==============================] - 0s 938us/step - loss: 0.0092 - accuracy: 0.9874
Epoch 165/200
179/179 [==============================] - 0s 934us/step - loss: 0.0113 - accuracy: 0.9818
Epoch 166/200
179/179 [==============================] - 0s 937us/step - loss: 0.0114 - accuracy: 0.9832
Epoch 167/200
179/179 [==============================] - 0s 937us/ste

CNNClassifier(batch_size=4, n_epochs=200, verbose=True)

In [27]:
network.score(ohlc_test, trend_test)

78/78 [==============================] - 0s 622us/step


0.8414239482200647

In [11]:
network.predict_proba(ohlc_test)

76/76 [==============================] - 0s 700us/step


array([[8.10072676e-21, 3.81799042e-02, 9.61820126e-01],
       [2.39082538e-02, 9.76091743e-01, 1.24802668e-16],
       [9.99096870e-01, 9.03171953e-04, 1.81466582e-12],
       [9.48157310e-01, 5.18427081e-02, 2.48431827e-18],
       [9.99944746e-01, 5.52708116e-05, 7.94344688e-23],
       [1.34985685e-16, 9.95744348e-01, 4.25562914e-03],
       [9.99942601e-01, 5.74048245e-05, 6.98000211e-15],
       [7.32752056e-17, 2.53798004e-04, 9.99746263e-01],
       [2.28354323e-15, 9.99999940e-01, 4.94050703e-08],
       [3.04217011e-01, 6.95783019e-01, 9.04828546e-15],
       [4.95415106e-02, 9.50458467e-01, 1.01190689e-13],
       [1.39994216e-09, 9.94732022e-01, 5.26795862e-03],
       [3.93378332e-06, 9.99996066e-01, 7.50782357e-14],
       [2.86800777e-16, 1.09749519e-04, 9.99890268e-01],
       [1.95947081e-09, 3.32747586e-02, 9.66725230e-01],
       [7.92874235e-24, 3.81129881e-04, 9.99618888e-01],
       [7.08022764e-18, 1.55504349e-05, 9.99984443e-01],
       [2.04499418e-04, 9.99795

### 4. 검증

In [29]:
from tools.instruments import instruments
from tools.datatools import norm
from tools.display import highchart
quotes = instruments.quotes(fields='ohlc')
symbols = list(quotes.columns.levels[0])

In [30]:
%load_ext autoreload
%aimport tools.datatools
%aimport tools.display
%autoreload 2

In [34]:
symbol = np.random.choice(symbols)
quote = quotes[symbol].dropna()
idx = 500#np.random.randint(0,len(quote)-51)
ohlc = quote.iloc[idx:idx+period]
data = norm(ohlc.values)
data = np.moveaxis(np.array([data]),1,2 )
print(network.predict_proba(data).round(2))
highchart([{'data':ohlc, 'type':'candlestick'}])

1/1 [==============================] - 0s 15ms/step
[[0.   0.19 0.81]]


### 3.모델을 파일로 저장

In [28]:
savepath = os.path.join('..','..','data','trend index','models', f'trend_{period}day')
network.save(savepath)

INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_100day\keras\assets


INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_100day\keras\assets


<zipfile.ZipFile filename='..\\..\\data\\trend index\\models\\trend_100day.zip' mode='r'>